In [51]:
import os
import random
import cv2
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split
import tensorflow
from tensorflow.python.client import device_lib

In [52]:
torch.cuda.is_available()

True

In [8]:
def seed_everything(seed: int = 24):
       random.seed(seed)
       np.random.seed(seed)
       # torch.cuda.manual_seed(seed)
       # torch.backends.cudnn.deterministic = True
seed_everything()

In [9]:
tot_actions = 5
actions_name = 'backward', 'sit', 'slide', 'swing', 'walk'
min_data_len = 45

In [ ]:
# discard unnecessary

for file in os.listdir('../csv_1031'):
       df = pd.read_csv(os.path.join('../csv_1031',file)

In [19]:
actions_csv_dir = '../csv_1031/'
dataset = []

label_mapping = {'backward': 0,
                 'sit': 1,
                 'slide': 2,
                 'swing': 3,
                 'walk' : 4
                 }

def map_action_to_label(csv_name):
       for action, label in label_mapping.items():
              if action in csv_name.split('_')[0]:
                     return label
       return -1

for action_csv in os.listdir(actions_csv_dir):
       action_df = pd.read_csv(os.path.join(actions_csv_dir, action_csv))
       label = map_action_to_label(action_csv)
       if label != -1:
              for idx in range(0, len(action_df), int(min_data_len / 2)):
                     seq_df = action_df[idx: idx + min_data_len] #길이만큼 데이터 자른 것(즉 length 만큼의 프레임)
                     if len(seq_df) == min_data_len: # 딱 length에 개수 맞춰서 끊어서 넣으려고
                            dataset.append({'key': label, 'value': seq_df}) # key에 slide, value에는 묶음 프레임 만큼이 담기겠네
       #최종적으로 dataset에는 행동별로 dictionary 가 만들어져 들어간다.

In [28]:
print(dataset[0]['value']) # z축 까지 99 (33 * 3)차원

      Nose_x    Nose_y    Nose_z  LEye_in_x  LEye_in_y  LEye_in_z    LEye_x  \
0   0.529178  0.469441  0.159390   0.521954   0.459262   0.131656  0.516659   
1   0.533761  0.471265  0.131279   0.526999   0.460200   0.107506  0.522267   
2   0.537234  0.473547  0.130224   0.530862   0.462024   0.105122  0.526176   
3   0.541347  0.473582  0.134560   0.534914   0.461973   0.107922  0.530146   
4   0.544573  0.473718  0.142126   0.537599   0.462001   0.109331  0.532539   
5   0.544895  0.473676  0.185405   0.537446   0.461991   0.141218  0.532060   
6   0.545285  0.473608  0.207851   0.537822   0.461922   0.161072  0.532282   
7   0.546466  0.471301  0.204747   0.539325   0.460225   0.158107  0.534083   
8   0.548138  0.470216  0.215264   0.541445   0.459510   0.166464  0.536139   
9   0.549583  0.470341  0.230958   0.543714   0.459808   0.182345  0.538515   
10  0.550495  0.471813  0.244705   0.545803   0.461264   0.197209  0.541062   
11  0.552844  0.473246  0.230610   0.548210   0.4624

In [53]:
class MyDataset(Dataset):
       def __init__(self, dataset): #모든 행동을 통합한 df가 들어가야함
              self.x = []
              self.y = []
              for dic in dataset:
                     self.y.append(dic['key']) #key 값에는 actions 들어감
                     self.x.append(dic['value']) #action마다의 data 들어감

       def __getitem__(self, index): #index는 행동의 index
              data = self.x[index] # x에는 꺼내 쓸 (행동마다 45개 묶음프레임)의 데이터
              label = self.y[index]
              return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

       def __len__(self):
              return len(self.x)


In [54]:
train_test_val_ratio = [0.7, 0.15, 0.15]
print(len(dataset))
train_len = int(len(dataset) * train_test_val_ratio[0])
val_len = int(len(dataset) * train_test_val_ratio[1])
test_len = len(dataset) - train_len - val_len
print('{}, {}, {}'.format(train_len, val_len, test_len))

664
464, 99, 101


In [55]:
CFG = {'batch_size': 8, 
       'learning_rate': 1e-4,
       'seed': 24,
       'epochs': 30
}

In [56]:
train_dataset = MyDataset(dataset)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=CFG['batch_size'])
val_loader = DataLoader(valid_data, batch_size=CFG['batch_size'])
test_loader = DataLoader(test_data, batch_size=CFG['batch_size'])

In [57]:
from sklearn.utils import shuffle

def shuffle_dataset(lm_list, label_list):
       lm_list, label_list = shuffle(lm_list, label_list, random_state=42)
       return lm_list, label_list

In [25]:
class Model(nn.Module):
       def __init__(self):
              super(Model, self).__init__()
              self.hidden_size=128
              self.lstm = nn.LSTM(input_size=99, hidden_size=128, batch_first=True)
              self.fc = nn.Linear(128, 5)
              # self.lstm1 = nn.LSTM(input_size=99, hidden_size=128, num_layers=1, batch_first=True) 
              # self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=1, batch_first=True)
              # self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=1, batch_first=True)
              # self.dropout1 = nn.Dropout(0, 1)
              # self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=1, batch_first=True)
              # self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True)
              # self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True)
              # self.dropout2 = nn.Dropout(0, 1)
              # self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1, batch_first=True)
              # self.fc = nn.Linear(32, 5) #분류할 클래스 5가지

       def forward(self, x):
              h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
              c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
              out, _ = self.lstm(0, (h0, c0))
              out = self.fc(out[:,-1, -1])
              out = nn.functional.softmax(out, dim=1)
              # x, _ = self.lstm1(x)
              # x, _ = self.lstm2(x)
              # x, _ = self.lstm3(x)
              # x = self.dropout1(x)
              # x, _ = self.lstm4(x)
              # x, _ = self.lstm5(x)
              # x, _ = self.lstm6(x)
              # x = self.dropout2(x)
              # x, _ = self.lstm7(x)
              # x = self.fc(x[:, -1, :])
              return x

In [26]:
class Model(nn.Module):
       def __init__(self):
              super(Model, self).__init__()

              ############## 1st To Do (20 points) ##############
              ###################################################
              self.recurrent_layer = nn.LSTM(hidden_size=100, input_size=99,  bidirectional=True)

              self.nonLin = nn.BatchNorm1d(45)
              self.recurrent_layer2 = nn.LSTM(hidden_size=100, input_size=200, bidirectional=True)
              self.nonLin2 = nn.BatchNorm1d(45)
              self.conv = nn.Conv1d(45,36,7,1)
              self.relu1 = nn.Softmax()
              self.classify_layer = nn.Linear(194, 5)
              ###################################################

       # the size of input is [batch_size, seq_len(15), input_dim(75)]
       # the size of logits is [batch_size, num_class]
       def forward(self, input, h_t_1=None, c_t_1=None):
              # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
              rnn_outputs, (hn, cn) = self.recurrent_layer(input)
              lin1 = self.nonLin(rnn_outputs)

              rnn_outputs2, (hn2, cn2) = self.recurrent_layer2(lin1)

              lin2 = self.nonLin2(rnn_outputs2)
              conv = self.conv(lin2)
              relu = self.relu1(conv)

              logits = self.classify_layer(relu[:,-1])
              return logits


In [58]:
class Model2(nn.Module):
       def __init__(self):
              super(Model2, self).__init__()
              self.lstm1 = nn.LSTM(input_size=99, hidden_size=128, num_layers=1, ) 
              self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=1,)
              self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=1,)
              self.dropout1 = nn.Dropout(0.5)
              self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=1,)
              self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1,)
              self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1,)
              self.dropout2 = nn.Dropout(0.5)
              self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1,)
              self.fc = nn.Linear(32, 5) #분류할 클래스 5가지

       def forward(self, x):
              x, _ = self.lstm1(x)
              x, _ = self.lstm2(x)
              x, _ = self.lstm3(x)
              x = self.dropout1(x)
              x, _ = self.lstm4(x)
              x, _ = self.lstm5(x)
              x, _ = self.lstm6(x)
              x = self.dropout2(x)
              x, _ = self.lstm7(x)
              x = self.fc(x[:, -1, :])
              return x

In [59]:
# for data, label in test_loader:
#        print('data:', len(data)) # 배치만큼 출력하니까 'data: .  8.. '
d = []
l = []
for data, label in test_loader:
       d.append(data)
       l.append(label)
print(d[0].shape) # 8개 잡은 배치, 45개 seqence 길이, 99 차원

torch.Size([8, 45, 99])


In [60]:
num_epochs = CFG['epochs']
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장

model = Model2()
optimizer = torch.optim.Adam(model.parameters(), lr=CFG['learning_rate'] )
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size = 5,
                                               gamma = 0.9) # learning rate scheduler 로 학습률 주기적 감소


val_acc_max = 0.74
val_loss_min = 0.3

for epoch in range(num_epochs):
       train_loss_list = []
       val_loss_list = []
       val_acc_list = []
       #모델 학습
       for i, (images, targets) in enumerate(train_loader):
              model.train()

              optimizer.zero_grad()
              outputs = model(images)

              targets = targets.long()
              loss = criterion(outputs, targets)
              loss.backward()
              optimizer.step()

              train_loss_list.append(loss.item())

              if (i+1) % 20 == 0:
                     print(f'Epoch: {epoch} - Loss: {loss:.6f}')

       train_loss = np.mean(train_loss_list)


       # 모델 검증
       for i, (images, targets) in enumerate(val_loader):
              model.eval()

              with torch.no_grad():
                     outputs = model(images)

                     targets = targets.long()

                     val_loss = criterion(outputs,targets)

                     preds = torch.argmax(outputs, dim=1)

                     batch_acc = (preds==targets).float().mean() # boolean 값의 평균

                     val_loss_list.append(val_loss.item())
                     val_acc_list.append(batch_acc.item())

       val_loss = np.mean(val_loss_list)
       val_acc = np.mean(val_acc_list)


       print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')


       #val 에서의 정확도 기준 모델 저장
       if val_acc_max < val_acc:
              valid_acc_max = val_acc
              best_models.append(model)
              print('model save, model val acc : ', val_acc)
           
           

       if val_loss_min > val_loss:
              val_loss_min = val_loss

lr_scheduler.step()
print('Train finished, best_models size : ',len(best_models))

Epoch: 0 - Loss: 1.626299
Epoch: 0 - Loss: 1.586203
Epoch: 0 - valid Loss: 1.566888 - valid_acc : 0.326923
Epoch: 1 - Loss: 1.537027
Epoch: 1 - Loss: 1.470422
Epoch: 1 - valid Loss: 1.456533 - valid_acc : 0.429487
Epoch: 2 - Loss: 1.449846
Epoch: 2 - Loss: 1.365402
Epoch: 2 - valid Loss: 1.423166 - valid_acc : 0.439103
Epoch: 3 - Loss: 1.371949
Epoch: 3 - Loss: 1.317908
Epoch: 3 - valid Loss: 1.428178 - valid_acc : 0.439103
Epoch: 4 - Loss: 1.418616
Epoch: 4 - Loss: 1.293901
Epoch: 4 - valid Loss: 1.434880 - valid_acc : 0.439103
Epoch: 5 - Loss: 1.437463
Epoch: 5 - Loss: 1.221160
Epoch: 5 - valid Loss: 1.439836 - valid_acc : 0.439103
Epoch: 6 - Loss: 1.352807
Epoch: 6 - Loss: 1.248215
Epoch: 6 - valid Loss: 1.443526 - valid_acc : 0.439103
Epoch: 7 - Loss: 1.397928
Epoch: 7 - Loss: 1.217305
Epoch: 7 - valid Loss: 1.446812 - valid_acc : 0.439103
Epoch: 8 - Loss: 1.405640
Epoch: 8 - Loss: 1.219226
Epoch: 8 - valid Loss: 1.450785 - valid_acc : 0.439103
Epoch: 9 - Loss: 1.341291
Epoch: 9 - 

KeyboardInterrupt: 

In [21]:
save_directory = "C:\\Users\\USER\Desktop\\19rne\\2023-RnE-main\saved_models/"


for i, model in enumerate(best_models):
       model_filename = f"best_model_fold_{i}.pth"
       model_path = save_directory + model_filename
       torch.save(model.state_dict(), model_path)
       print('asdf')

print(f"Saved {len(best_models)} models to {save_directory}")


asdf
asdf
asdf
asdf
asdf
asdf
asdf
asdf
Saved 8 models to C:\Users\USER\Desktop\19rne\2023-RnE-main\saved_models


In [21]:
save_directory = "C:\\Users\\USER\Desktop\\19rne\\2023-RnE-main\saved_models/"

In [ ]:
import matplotlib.pyplot as plt

# Validation accuracy values
val_acc_values = [0.708333, 0.777778, 0.75, 0.722222, 0.680556, 0.708333,
                  0.736111, 0.708333, 0.75, 0.708333, 0.736111, 0.791667,
                  0.666667, 0.625, 0.680556, 0.722222, 0.763889, 0.819444,
                  0.777778, 0.75]

# Plotting the validation accuracy over epochs
plt.figure(figsize=(10, 5))
plt.plot(val_acc_values, label='Validation Accuracy', marker='o')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# 정확도 검증
for m in best_models:
       with torch.no_grad():
              test_loss, test_acc = epoch(test_loader, mode='test')
              test_acc = round(test_acc, 4)
              test_loss = round(test_loss, 4)
              print('Test Acc.: {}'.format(test_acc))
              print('Test Loss: {}'.format(test_loss))


In [61]:
def load_model(model, filepath):
       model.load_state_dict(torch.load(filepath))
       model.eval()
       return model


In [62]:
load_directory = save_directory


num_models = 8


test_accuracies = []

# Iterate through each model
for i in range(num_models):
       # Create a new instance of your model
       model = Model()

       # Load the model from the file
       model_filename = f"best_model_fold_{i}.pth"
       model_path = load_directory + model_filename
       model = load_model(model, model_path)

       # Test the model on the test set
       correct = 0
       total = 0
       with torch.no_grad():
              for images, targets in test_loader:
                     outputs = model(images)
                     _, predicted = torch.max(outputs.data, 1)
                     total += targets.size(0)
                     correct += (predicted == targets).sum().item()

       # Calculate the accuracy for this model
       accuracy = correct / total
       test_accuracies.append(accuracy)

       print(f"Model {i+1} - Test Accuracy: {accuracy:.4f}")

# Print the average test accuracy across all models
average_accuracy = sum(test_accuracies) / num_models
print(f"Average Test Accuracy: {average_accuracy:.4f}")


NameError: name 'save_directory' is not defined

In [49]:
# 여기부터 실시간 영상 테스트

interval = 1
video_path = r'C:\Users\USER\Desktop\19rne\video_data\swing_17.mp4'

cap = cv2.VideoCapture(video_path)
img_list = []
if cap.isOpened():
       cnt = 0
       while True:
              ret, img = cap.read()
              if ret:
                     img = cv2.resize(img, (640, 640))
                     if cnt == interval:
                            img_list.append(img)
                            cnt = 0
                     cv2.imshow(video_path, img)
                     cv2.waitKey(1)
                     cnt += 1
              else:
                     break
cap.release()
cv2.destroyAllWindows()

print('저정된 frame의 개수: {}'.format(len(img_list)))

저정된 frame의 개수: 647


In [64]:
# 연속 시퀀스 분석
from tqdm import tqdm
model_filename = "best_model_fold_4.pth"
model_path = '../saved_models/' + model_filename
model = load_model(Model(), model_path)

model.eval()
out_img_list = []
dataset = []
status = 'None'
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)


print('시퀀스 데이터 분석 중...')
xy_list_list = []


for img in tqdm(img_list):
       results = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
       if not results.pose_landmarks: continue
       
       xy_list = []
       idx = 0
       for x_and_y in results.pose_landmarks.landmark:
              xy_list.append(x_and_y.x)
              xy_list.append(x_and_y.y)
              x, y = int(x_and_y.x * 640), int(x_and_y.y * 640)
              idx += 1
       xy_list_list.append(xy_list)

       length = 45
       if len(xy_list_list) == length:
              dataset = []
              dataset.append({'key': 4, 'value': xy_list_list})
              dataset = MyDataset(dataset)
              dataset = DataLoader(dataset)
              xy_list_list = []
              for data, label in dataset:
                     with torch.no_grad():
                            result = model(data)
                            _, out = torch.max(result, 1)
                            print(out.item())
              
              if out.item() == 0:
                     status = 'backward'
              elif out.item() == 1:
                     status = 'sit'
              elif out.item() == 2:
                     status = 'slide'
              elif out.item() == 3:
                     status = 'swing'
              elif out.item() == 4:
                     status = 'walk'
               
       cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 2)
       #cv2.imshow('asdf', img)
       out_img_list.append(img)

       

시퀀스 데이터 분석 중...


  8%|▊         | 49/647 [00:01<00:18, 33.16it/s]

2


 14%|█▍        | 93/647 [00:02<00:16, 33.00it/s]

2


...

 91%|█████████ | 589/647 [00:17<00:01, 32.93it/s]

2


 98%|█████████▊| 637/647 [00:19<00:00, 33.20it/s]

2


100%|██████████| 647/647 [00:19<00:00, 33.10it/s]


In [21]:
for img in out_img_list:
       cv2.imshow('asdf', img)